In [1]:
import json
a = open('data.json')
data = json.load(a)


def read(value, bagage = []):
    for key, value in value.items():
        classes.append(key)
        cur_bagage = bagage + [key]
        if type(value) is dict:
            read(value, cur_bagage)
        elif type(value) is list:
            for i in value:
                classes.append(i)
                entries.append(cur_bagage + [i])

classes = []
entries = []
y = []
X_subject = []

read(data)
classes = classes[1:]
for animal in entries:
    X_subject.append(animal[-1])
    y.append([1 if encoding in animal else 0 for encoding in classes])

In [2]:
from threading import Thread
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import requests
import numpy as np
from tqdm import tqdm
from PIL import Image
import cv2
import os
from requests.auth import HTTPProxyAuth


MAX_THEADS = 10

PROXY = '66.23.232.83:3128'
session = requests.Session()
session.trust_env = False
session.proxies = {
    "http": f'http://{PROXY}',
    "https": f'http://{PROXY}'
}
# session.auth = HTTPProxyAuth('', '')

webdriver.DesiredCapabilities.CHROME['proxy'] = {
    "httpProxy": PROXY,
    "ftpProxy": PROXY,
    "sslProxy": PROXY,
    "proxyType": "MANUAL",
}

def get(word):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('headless')
    chrome_options.add_argument('window-size=1920x1080')
    chrome_options.add_argument("disable-gpu")
    driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)
    driver.get(f'https://duckduckgo.com/?q=animal+{word}&iax=images&ia=images')
    links = []
    old = 0
    length = -1
    while True:
        try:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            driver.implicitly_wait(30)
            time.sleep(4)
            element = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".tile--img__img")))
            old = length
            length = len(element)
            print(f'itens found: {length}')
            if length == old:
                links = [i.get_attribute('src') for i in element]
                break
        except TimeoutException:
                print("Loading took too much time!")
    path = f'animals/{word}'
    c = ''
    for d in path.split('/'):
        # handle instances of // in string
        if not d: continue 

        c += d + '/'
        if not os.path.isdir(c):
            os.mkdir(c)
    

    def resize(img, size):
        img = np.array(img)
        if(img.shape[1]<img.shape[0]):
            w = int(img.shape[1]*size/img.shape[0])
            h = size
        else:
            w = size
            h = int(img.shape[0]*size/img.shape[1])
        return cv2.resize(img,(w,h))
    key = 0
    for link in tqdm(links):
        with Image.open(session.get(link, stream = True).raw) as image:
                cv2.imwrite(os.path.join(f'{path}/{key}.jpg'), cv2.cvtColor(resize(image, 500), cv2.cv2.COLOR_BGR2RGB))
        key += 1

threads = []
for animal in X_subject:
    while len(threads) >= MAX_THEADS:
        for j in threads:
            if not j.is_alive():
                threads.remove(j)
    thread = Thread(target = get, args = [animal])
    thread.start()
    threads.append(thread)

C:\Users\THIAGO\AppData\Local\Temp/ipykernel_58376/753679225.py:41: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)


itens found: 100
itens found: 100
itens found: 188
itens found: 173
itens found: 261
itens found: 248
